In [ ]:
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.spatial.distance
import scipy.ndimage.filters
import copy
from matplotlib import animation
import matplotlib as mpl
from pylab import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import font_manager
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.font_manager as fm
from matplotlib.pyplot import cm

mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['font.family'] = 'Sans'
mpl.rcParams['font.size'] = 16
mpl.rcParams['axes.linewidth'] = 2

%matplotlib inline

## Read colvars and bias from OPES runs biasing projection and extension along and from axis between A1-A2 and A4-A5

In [ ]:
%%bash
bash utils/get_bias.sh holo.readbias.dat

In [ ]:
workdir = "outputs/holo_pn/" #directory where colvars and biases are 
figure_dir = 'test_figs'
data_dir = 'test_data'
os.makedirs(figure_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
workdir

In [ ]:
cv_bias_data = {}
walls_data = {}

for i in np.linspace(0.0,0.0,1):
    if round(i,3) == -0.0:
        key = "0.0"
    else:
        key = str(round(i,3))
    cv_bias_data[key]=[]
    walls_data[key]=[]

file_template = "_FORCE/DIR/DIR_fes_pullVt_rerun_WIN.colvar_bias.out"

for key in cv_bias_data:
    file_name = file_template.replace('FORCE', key)
    for run in range(26,46):
        run_name = file_name.replace('DIR',str(run))
        final_name=run_name.replace('WIN', str(26))
        dist = np.genfromtxt(workdir+final_name, comments='#', dtype=float, usecols = (0,1,2,3,4), skip_header=1501) #skip transient
        biases = dist[::25]
        for win in range(27,46):
            print(run,win,key)
            final_name=run_name.replace('WIN', str(win))
            bias_win = np.genfromtxt(workdir+final_name, comments='#', dtype=float, usecols = (4), skip_header=1501)
            biases = np.hstack((biases,np.reshape(bias_win[::25],(-1,1))))
            
        cv_bias_data[key].append(biases)
        


In [ ]:
file_template =  "_FORCE/DIR/DIR_fes_pullVt.walls.out"
for key in cv_bias_data:
    file_name = file_template.replace('FORCE', key)
    temp = []
    for run in range(26,46):
        final_name = file_name.replace('DIR',str(run))
        dist = np.genfromtxt(workdir+final_name, comments='#', dtype=float, skip_header=1501)
        walls = dist[:,1:][::25]
        total_walls = np.sum(walls,axis=1)
        outcolumn = np.vstack((dist[:,0][::25], total_walls))
        temp.append(outcolumn.transpose())
    walls_data[key] = temp

In [ ]:
kbT = 0.596161
barrier = 25 # BARRIER parameter used in OPES runs

concat_cvs_bias = {}
concat_walls = {}
concat_walls_time ={}

for key in cv_bias_data:
    biases = cv_bias_data[key]
    concatenated_opes_bias = biases[0]
    walls = walls_data[key]
    concatenated_wall_bias = walls[0][:,1][::]
    cat_wall_time = walls[0][::]

    cat_pbias = pull_biases[0]
    for i in range(1, len(biases)):
        bias = biases[i]
        wall_bias = walls[i]
        concatenated_opes_bias = np.concatenate((concatenated_opes_bias,bias), axis=0)
        concatenated_wall_bias = np.concatenate((concatenated_wall_bias, wall_bias[:,1][::]), axis=0)
        cat_wall_time = np.concatenate((cat_wall_time, wall_bias[::]), axis=0)
        
    concat_cvs_bias[key] = concatenated_opes_bias
    concat_walls[key] = concatenated_wall_bias
    concat_walls_time[key] = cat_wall_time


In [ ]:
from utils.wham import wham
wham_weights = {}
for key in concat_cvs_bias:
    concatenated_bias=concat_cvs_bias[key][:,4:]
    print(f"force: {key}")
    w=wham(concatenated_bias,T=kbT, verbose=False)
    wham_weights[key] = w['logW']


In [ ]:
for key in concat_cvs_bias:
    cvs = concat_cvs_bias[key][:,:4]
    walls_t = concat_walls_time[key]
    wham_bias = wham_weights[key]*kbT
    pbiases = concat_pbias[key]
    filename = f'data_highE/COLVAR_cat_{key}.dat'
    head=" FIELDS time proj ext cmap"
    np.savetxt(filename, cvs, delimiter=' ', newline='\n', header=head, comments='#!', fmt='%.6e')
    filename = f'data_highE/WALL_cat_{key}.dat'
    head=" FIELDS time totalwall.bias"
    np.savetxt(filename, walls_t, delimiter=' ', newline='\n', header=head, comments='#!', fmt='%.6e')
    wham_bias_t = np.hstack((cvs[:,0].reshape(-1,1),wham_bias.reshape(-1,1)))
    filename = f'data_highE/BIAS_cat_{key}.dat'
    head=" FIELDS time wham.bias"
    np.savetxt(filename, wham_bias_t, delimiter=' ', newline='\n', header=head, comments='#!', fmt='%.6e')
    pbias_t = np.hstack((cvs[:,0].reshape(-1,1),pbiases.reshape(-1,1)))
    filename = f'data_highE/PBIAS_cat_{key}.dat'
    head=" FIELDS time p.bias"
    np.savetxt(filename, pbias_t, delimiter=' ', newline='\n', header=head, comments='#!', fmt='%.6e')
     

In [ ]:
%%bash
bash utils/reweight_cat.sh plumed_rw_cat_cmap.dat

In [ ]:
all_fes_ext_proj = {}
all_fes_cnum_proj = {}
all_fes_cnum_ext = {}
all_fes_proj_cmap = {}
all_fes_ext_cmap = {}

all_fes_ext = {}
all_fes_proj = {}
all_fes_cnum = {}
all_fes_cmap = {}

for i in np.linspace(0.0,0.0,1):
    if round(i,3) == -0.0:
        key = "0.0"
    else:
        key = str(round(i,3))
    all_fes_ext_proj[key]=[]
    all_fes_cnum_proj[key]=[]
    all_fes_cnum_ext[key] = []
    all_fes_ext[key]=[]
    all_fes_proj[key]=[]
    all_fes_cnum[key] =[]
    all_fes_cmap[key] = []
    all_fes_proj_cmap[key] = []
    all_fes_ext_cmap[key] = []


file_template = "data_highE/FFORCE_fes_projext_rw.dat"
for k in all_fes_ext_proj:
    final_name = file_template.replace('FORCE', k)
    dist = np.genfromtxt(final_name, comments='#',dtype=float)
    all_fes_ext_proj[k].append(dist)

    
file_template = "data_highE/FFORCE_fes_projcmap_rw.dat"
for k in all_fes_proj_cmap:
    final_name = file_template.replace('FORCE', k)
    dist = np.genfromtxt(final_name, comments='#',dtype=float)
    all_fes_proj_cmap[k].append(dist)

file_template = "data_highE/FFORCE_fes_extcmap_rw.dat"
for k in all_fes_ext_cmap:
    final_name = file_template.replace('FORCE', k)
    dist = np.genfromtxt(final_name, comments='#',dtype=float)
    all_fes_ext_cmap[k].append(dist)
        
file_template = "data_highE/FFORCE_fes_proj_rw.dat"
for k in all_fes_proj:
    final_name = file_template.replace('FORCE', k)
    dist = np.genfromtxt(final_name, comments='#',dtype=float)
    all_fes_proj[k].append(dist)
        
file_template = "data_highE/FFORCE_fes_ext_rw.dat"
for k in all_fes_ext :
    final_name = file_template.replace('FORCE', k)
    dist = np.genfromtxt(final_name, comments='#',dtype=float)
    all_fes_ext[k].append(dist)
      
file_template = "data_highE/FFORCE_fes_cmap_rw.dat"
for k in all_fes_cmap:
    final_name = file_template.replace('FORCE', k)
    dist = np.genfromtxt(final_name, comments='#',dtype=float)
    all_fes_cmap[k].append(dist)

In [ ]:
def plot_energy_path(unit, model, forces_barrier, outdir):
    """
    plot energy path
    Args:
        distances:
        energies:
        unit:

    Returns:

    """
    plt.figure(figsize=(5, 4))
    for k in forces_barrier:
        p = forces_barrier[k]
        distances = p[:,0] 
        energies = p[:,1]
        max_energy = max(energies)
        min_energy = min(energies)
        max_ind = np.argmax(energies)
        energy_range = max_energy - min_energy
        plt.plot(distances, energies, 'o-', markerfacecolor='w', label=k)
        plt.xlim([0, 1])
        plt.ylim([min_energy, max_energy + 0.1 * energy_range])
        plt.plot([0, distances[max_ind]], [max_energy, max_energy], '--', color='r')
        #plt.text(distances[max_ind], max_energy, '$E_a=%.2f$ %s' % (max_energy, unit))
        plt.xlabel('Reaction Coordinates')
        plt.ylabel('$E$ (%s)' % unit)
    plt.legend(fontsize=12)
    plt.savefig(f'{outdir}/{model}_epath_force.png')
    plt.show()
    plt.close()

def plot_energy_path_str(unit, model, forces_barrier, outdir):
    """
    plot energy path
    Args:
        distances:
        energies:
        unit:

    Returns:

    """
    fcolors =  cm.viridis(np.linspace(0, 1, len(forces_barrier)))
    plt.figure(figsize=(5, 4))
    for i,k in enumerate(forces_barrier):
        p = forces_barrier[k]
        distances = p[0]/(len(p[0])-1) 
        energies = p[1]
        max_energy = max(energies)
        min_energy = min(energies)
        max_ind = np.argmax(energies)
        energy_range = max_energy - min_energy
        if float(k)<0:
            f_label = f'{np.abs(float(k))} pN (+)'
        elif float(k)>0:
            f_label = f'{np.abs(float(k))} pN (-)'

        
        plt.xlim([0, 1])
        plt.ylim([min_energy, max_energy + 0.1 * energy_range])
        if k == '0.0':
            plt.plot([0, distances[max_ind]], [max_energy, max_energy], '--', color='r', linewidth=1.0)
            plt.plot(distances, energies, 'o-', markerfacecolor='w', label=f'{k} pN', color='r')
        else:
            plt.plot(distances, energies, 'o-', markerfacecolor='w', label=f_label, color=fcolors[i])
            plt.plot([0, distances[max_ind]], [max_energy, max_energy], '--', color=fcolors[i], linewidth=1.0)
        #plt.text(distances[max_ind], max_energy, '$E_a=%.2f$ %s' % (max_energy, unit))
        plt.xlabel(r'$rc$')
        plt.ylabel(r'$\Delta E \ (%s)$' % unit)
    plt.legend(fontsize=10, loc='lower right')
    plt.savefig(f'{outdir}/{model}_epath_force_str.pdf', bbox_inches='tight', format='pdf')
    plt.show()
    plt.close()

In [ ]:
from utils.stringbase import *

barrier=25
force_barriers_str = {}

file_template = "data_highE/FFORCE_fes_projext_rw.dat"
for key in all_fes_ext_proj:
    runs=all_fes_ext_proj[key]
    final_name = file_template.replace('FORCE', key)
    for i,run in enumerate(runs):
        fig,ax = plt.subplots(figsize=(5, 4))
        x = run[:,0]
        y = run[:,1]
        z = run[:,2]
        index=np.where((z>=barrier))[0]
        z[index]=barrier
        X = np.linspace(np.amin(x),np.amax(x), 101)
        Y = np.linspace(np.amin(y), np.amax(y), 101)
        XZ = z.reshape(101,101)
        #fes_twocv.append(XZ)
        ax.contour(X, Y, XZ, 10, colors='black', linewidths=0.6)
        im=ax.imshow(XZ,aspect='auto',origin='lower', cmap='jet', interpolation="gaussian", extent=(x.min(), x.max(), y.min(), y.max()))
        ax.set_title(r"$apo$", size=14)
        ax.set_xlabel(r'$projection\ (\AA)$', size=16)
        ax.set_ylabel(r'$extension\ (\AA)$', size=16)
        # ax.set_xticks([i for i in range(40,91,5)])
        divider = make_axes_locatable(ax)
        cax2 = divider.append_axes("right", size="5%", pad=0.05)
        cbar=fig.colorbar(im, ax=ax, cax=cax2)
        cbar.ax.tick_params(labelsize=16)
        cbar.set_label(r"$FE\ (kcal/mol)$")
        
        landscape = Landscape.from_plumed(final_name,barrier)
        
        if float(key)< 0:
            x_string=np.linspace(71,71,20) #Initial guess for the position of nodes of the string
            y_string=np.linspace(40.,46,20)

        elif float(key)==0:
            x_string=np.linspace(72.5,70,20) #Initial guess for the position of nodes of the string
            y_string=np.linspace(40.5,46.0,20)
    
        else:
            x_string=np.linspace(72.5,70,20) #Initial guess for the position of nodes of the string
            y_string=np.linspace(40.5,46,20)
        
        string = String(x_string,y_string)
        string.reparameterize(len(x_string))
        np.random.seed(5)
        
        drift_factor = 0.001
        sigma = 0.01 #for random walk
        N = 25
        t = 0

        string_history = [(string.xdata.copy(),string.ydata.copy())]

        for t in range(4000):
            string.random_walk(sigma,sigma)
            string.drift(landscape,drift_factor)
            string.untangle()
            string.reparameterize(N)
            string_history.append((string.xdata.copy(),string.ydata.copy()))

        
        idx_point = np.arange(0,len(string.get_pmf(landscape)),1)
        data_min = [idx_point,string.get_pmf(landscape)-min(string.get_pmf(landscape))]
        force_barriers_str[key]=data_min
    
        ax.plot(string.xdata,string.ydata, 'o-', markerfacecolor='w', c='r')
        fig.savefig(F"{figure_dir}/fes_extproj_rw_{key}_plumed_wham_str.png", bbox_inches='tight')
        plt.show()       